In [1]:
import pandas as pd
from sodapy import Socrata
from IPython.display import display
import gmplot
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from config import gkey
import re
import numpy as np
import gmaps
%matplotlib inline

Since we are looking at county level data in Connecticut, caching some county information. 
1. Use BeautifulSoup to get the list of counties in Connecticut from Wikipedia
2. Use Google geocode to get latitude and longitude information for each county.
This information will be used later to generate a heatmap.

In [2]:
wiki = "https://en.wikipedia.org/wiki/List_of_counties_in_Connecticut"
req = requests.get(wiki)
soup = BeautifulSoup(req.text,"lxml")
table = soup.find("table", { "class" : "wikitable sortable" })

In [3]:
counties_list = []
for row in table.findAll("tr"):
    cells = row.findAll("td")
    if cells:
        county = cells[0].find(text=True)
        county = re.sub("County","",county).upper().rstrip()
        counties_list.append(county)
counties_list

['FAIRFIELD',
 'HARTFORD',
 'LITCHFIELD',
 'MIDDLESEX',
 'NEW HAVEN',
 'NEW LONDON',
 'TOLLAND',
 'WINDHAM']

In [4]:
geo_location = {county:{} for county in counties_list}
for county in counties_list:
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?" \
            f"address={county} county,CT&key={gkey}"
    try:
        geo_data = requests.get(target_url).json() 
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    except Exception as exp:
        print(f"ERROR2:{exp}, {geo_data}")
        lat = ""
        lng = ""
    geo_location[county]["lat"] = lat
    geo_location[county]['lng'] = lng
    
pprint(geo_location)

{'FAIRFIELD': {'lat': 41.2559654, 'lng': -73.37086959999999},
 'HARTFORD': {'lat': 41.7924343, 'lng': -72.8042797},
 'LITCHFIELD': {'lat': 41.7866294, 'lng': -73.2765361},
 'MIDDLESEX': {'lat': 41.46985050000001, 'lng': -72.4731529},
 'NEW HAVEN': {'lat': 41.3266911, 'lng': -72.8042797},
 'NEW LONDON': {'lat': 41.5185189, 'lng': -72.0468164},
 'TOLLAND': {'lat': 41.8418987, 'lng': -72.37846789999999},
 'WINDHAM': {'lat': 41.8275622, 'lng': -72.0468164}}


Getting Drug Death related information from CT, and cleaning up (removing rows which do not have county information.
TODO: Get county information from the city information

In [5]:
client = Socrata("data.ct.gov", None)

In [6]:
results = client.get("u2qy-687x")

In [7]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [8]:
results_df.head()

,:@computed_region_dam5_q64j,:@computed_region_m4y2_whse,:@computed_region_nhmp_cq6b,:@computed_region_snd5_k6zv,age,amphet,any_opioid,benzo_s,casenumber,coc,...,morphine_not_heroin,other,oxyc,oxym,race,residence_city,residence_county,residence_state,sex,tramad
0,1045,59,243,8,53,NaN,NaN,Y,13-16336,NaN,...,NaN,NaN,Y,Y,White,GROTON,NEW LONDON,NaN,Female,NaN
1,1044,151,167,4,30,NaN,NaN,NaN,12-18447,Y,...,NaN,NaN,Y,NaN,White,WOLCOTT,NEW HAVEN,NaN,Male,NaN
2,1041,49,108,1,43,NaN,NaN,Y,14-2758,NaN,...,NaN,NaN,NaN,NaN,White,ENFIELD,NaN,NaN,Male,NaN
3,1044,148,195,7,24,NaN,NaN,NaN,14-13497,NaN,...,NaN,NaN,NaN,NaN,White,WALLINGFORD,NaN,NaN,Female,NaN
4,1044,156,11,7,26,NaN,NaN,NaN,13-14421,NaN,...,NaN,NaN,NaN,NaN,White,WEST HAVEN,NEW HAVEN,NaN,Female,NaN


In [9]:
results_df.columns

Index([':@computed_region_dam5_q64j', ':@computed_region_m4y2_whse',
       ':@computed_region_nhmp_cq6b', ':@computed_region_snd5_k6zv', 'age',
       'amphet', 'any_opioid', 'benzo_s', 'casenumber', 'coc', 'date',
       'death_city', 'death_county', 'death_state', 'deathloc',
       'deathloc_city', 'deathloc_state', 'etoh', 'fentanyl', 'heroin',
       'hydr_cod', 'immediatecausea', 'injuryplace', 'location',
       'mannerofdeath', 'methadone', 'morphine_not_heroin', 'other', 'oxyc',
       'oxym', 'race', 'residence_city', 'residence_county', 'residence_state',
       'sex', 'tramad'],
      dtype='object')

In [10]:
results_df = results_df.dropna(subset=['death_county'])

In [12]:
target_url = f"https://maps.googleapis.com/maps/api/geocode/json?" \
            f"address=Hartford,CT,USA&key={gkey}"
try:
    geo_data = requests.get(target_url).json() 
    lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    lng = geo_data["results"][0]["geometry"]["location"]["lng"]
except:
    print ("Cannot find geocode for Hartford")

Generate a google heatmap for the density of deaths by location.

In [15]:
gmap = gmplot.GoogleMapPlotter(lat,lng,10)

In [16]:
gmap

In [17]:
def get_geocode(row):
    place = row["death_county"]
    try:
        row["lat"] = geo_location[place]["lat"]
        row['lng'] = geo_location[place]["lng"]
    except:
        print (f"Cannot find {place} in CT counties, {row.name}.")
        row["lat"] = np.nan
        row["lng"] = np.nan
    return row

In [18]:
results_df = results_df.apply(get_geocode,axis=1)
#results_df["lng"] = results_df.apply(get_geolng, axis=1)
results_df = results_df.dropna(subset=['lat','lng'])

Cannot find USA in CT counties, 243.


In [19]:
gmap.heatmap(results_df["lat"], results_df["lng"])
gmap.draw("my_heatmap.html")

In [20]:
results_df[results_df["death_county"] == "USA"]

,:@computed_region_dam5_q64j,:@computed_region_m4y2_whse,:@computed_region_nhmp_cq6b,:@computed_region_snd5_k6zv,age,amphet,any_opioid,benzo_s,casenumber,coc,...,oxyc,oxym,race,residence_city,residence_county,residence_state,sex,tramad,lat,lng


In [21]:
zip(results_df.lat, results_df.lng)

In [22]:
gmap_data = list(zip(results_df.lat, results_df.lng))


In [23]:
gmaps.configure(api_key=gkey)
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(gmap_data)
#heatmap_layer.gradient = ['white','silver','gray']
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'